Test the PAC pool function

In [1]:
import pacnet.pac as pac

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torch.nn.modules.utils import _pair
import os
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import time
from functools import reduce
from utils import plot
from data.dataset import ElasticityDataSet

In [2]:
temp = torch.randint(0, 9, (2, 2, 5, 5)).float()
temp[0, 0]

tensor([[8., 3., 6., 5., 4.],
        [8., 0., 2., 1., 8.],
        [5., 7., 3., 7., 7.],
        [4., 4., 3., 3., 8.],
        [2., 8., 7., 1., 0.]])

In [3]:
sac = pac.PacPool2d(out_channels=2, kernel_size=3, padding=1, kernel_type='quad')
pool = nn.AvgPool2d(3, 1, padding=1)


In [4]:
def gen_kernels(a):
    #k1 = 1./9.*torch.asarray([[1,1,1],[1,1,1],[1,1,1]], dtype=torch.float32)
    #k2 = 1./9.*torch.asarray([[1,1,1],[1,1,1],[1,1,1]], dtype=torch.float32)
    k1 = torch.asarray([[0,1,0],[0,2,0],[0,1,0]], dtype=torch.float32)
    k2 = torch.asarray([[0,0,0],[1,2,1],[0,0,0]], dtype=torch.float32)
    b, ch, h, w = a.shape
    out = torch.zeros(size=(b, ch, 3, 3, h, w))
    for i in range(h):
        for j in range(w):
            if ((i+j)%2 != 0):
                out[:, :, :, :, i, j] = k1 * (i+j)
            else:
                out[:, :, :, :, i, j] = k2 * (i+j)
    return out

In [5]:
kernels = gen_kernels(temp)

In [6]:
out_pool = pool(temp)
out_pool

tensor([[[[2.1111, 3.0000, 1.8889, 2.8889, 2.0000],
          [3.4444, 4.6667, 3.7778, 4.7778, 3.5556],
          [3.1111, 4.0000, 3.3333, 4.6667, 3.7778],
          [3.3333, 4.7778, 4.7778, 4.3333, 2.8889],
          [2.0000, 3.1111, 2.8889, 2.4444, 1.3333]],

         [[1.0000, 2.0000, 3.2222, 4.1111, 3.1111],
          [1.4444, 2.5556, 4.3333, 5.5556, 4.4444],
          [2.3333, 3.7778, 4.5556, 5.0000, 3.5556],
          [2.4444, 3.2222, 4.2222, 4.1111, 3.3333],
          [2.0000, 2.6667, 3.1111, 2.6667, 2.0000]]],


        [[[1.7778, 3.3333, 3.0000, 2.6667, 1.1111],
          [2.6667, 4.2222, 4.3333, 3.2222, 1.6667],
          [2.6667, 4.1111, 3.3333, 2.4444, 1.0000],
          [2.2222, 3.5556, 4.1111, 3.3333, 2.0000],
          [1.3333, 2.6667, 2.7778, 2.7778, 1.4444]],

         [[1.4444, 2.6667, 2.7778, 2.8889, 1.6667],
          [2.5556, 4.6667, 4.2222, 4.6667, 2.5556],
          [2.4444, 4.3333, 3.3333, 4.5556, 2.6667],
          [3.2222, 4.7778, 3.5556, 4.8889, 3.3333],
    

In [7]:
out_sac = sac(temp, None, kernels)
out_sac

tensor([[[[  0.,   6.,  40.,  33.,  52.],
          [ 29.,  20.,  39.,  48., 135.],
          [ 34.,  54.,  80.,  90., 126.],
          [ 45.,  60.,  80., 102., 161.],
          [ 48., 100., 138.,  35.,   8.]],

         [[  0.,  10.,  32.,  60.,  92.],
          [  5.,  30.,  48., 100., 135.],
          [ 12.,  39.,  44., 120., 102.],
          [ 51.,  92.,  65., 102.,  98.],
          [ 28.,  75.,  78., 140., 112.]]],


        [[[  0.,   9.,  56.,  48.,  32.],
          [ 19.,  32.,  60.,  32.,  25.],
          [ 16.,  63.,  48.,  40.,  36.],
          [ 18.,  72.,  95.,  54.,  84.],
          [ 20.,  70., 114.,  56., 144.]],

         [[  0.,  16.,  46.,  33.,  60.],
          [ 12.,  16.,  75.,  48., 105.],
          [ 30.,  42.,  84.,  20.,  96.],
          [ 75.,  68.,  85., 102., 210.],
          [ 68.,  30.,  72.,  91., 136.]]]])

In [8]:
# test group_net
group_net = nn.Conv2d(in_channels=4, out_channels=2, kernel_size=1, groups=2, bias=False)
group_net.weight.data.fill_(1.0)

tensor([[[[1.]],

         [[1.]]],


        [[[1.]],

         [[1.]]]])

In [9]:
temp2 = torch.randint(0, 9, (1, 4, 5, 5)).float()
temp2[0,:,:,:]

tensor([[[5., 6., 0., 0., 1.],
         [1., 4., 3., 4., 6.],
         [1., 6., 1., 0., 8.],
         [6., 8., 7., 3., 6.],
         [3., 4., 4., 2., 8.]],

        [[1., 1., 1., 7., 8.],
         [2., 8., 6., 3., 5.],
         [7., 3., 5., 6., 2.],
         [0., 5., 5., 0., 4.],
         [7., 6., 0., 3., 2.]],

        [[7., 1., 2., 1., 5.],
         [2., 1., 8., 5., 6.],
         [8., 1., 1., 3., 8.],
         [0., 6., 3., 8., 7.],
         [6., 7., 8., 7., 6.]],

        [[2., 6., 2., 8., 1.],
         [7., 2., 1., 7., 7.],
         [7., 4., 1., 5., 1.],
         [1., 3., 4., 3., 2.],
         [1., 5., 3., 5., 3.]]])

In [10]:
group_net(temp2)

tensor([[[[ 6.,  7.,  1.,  7.,  9.],
          [ 3., 12.,  9.,  7., 11.],
          [ 8.,  9.,  6.,  6., 10.],
          [ 6., 13., 12.,  3., 10.],
          [10., 10.,  4.,  5., 10.]],

         [[ 9.,  7.,  4.,  9.,  6.],
          [ 9.,  3.,  9., 12., 13.],
          [15.,  5.,  2.,  8.,  9.],
          [ 1.,  9.,  7., 11.,  9.],
          [ 7., 12., 11., 12.,  9.]]]], grad_fn=<ConvolutionBackward0>)

In [11]:
# test f_net()
f_net = nn.Conv2d(in_channels=2,out_channels=2,kernel_size=3, padding=1, groups=2, bias=False)

In [12]:
temp[0]

tensor([[[8., 3., 6., 5., 4.],
         [8., 0., 2., 1., 8.],
         [5., 7., 3., 7., 7.],
         [4., 4., 3., 3., 8.],
         [2., 8., 7., 1., 0.]],

        [[1., 3., 3., 7., 8.],
         [1., 4., 6., 6., 7.],
         [2., 2., 1., 7., 5.],
         [7., 5., 6., 4., 3.],
         [1., 5., 0., 8., 3.]]])

In [13]:
h = 0.1
f_weights = torch.asarray([ [h*h/36., h*h/9., h*h/36.],
                            [h*h/9., 4.*h*h/9., h*h/9.],
                            [h*h/36., h*h/9., h*h/36.]], dtype=torch.double).reshape(1,1,3,3)
f_net.weight.data[:] = f_weights
out = f_net(temp)

In [14]:
out[0]

tensor([[[0.0478, 0.0317, 0.0381, 0.0372, 0.0325],
         [0.0528, 0.0283, 0.0261, 0.0344, 0.0522],
         [0.0444, 0.0492, 0.0367, 0.0525, 0.0578],
         [0.0342, 0.0469, 0.0386, 0.0392, 0.0489],
         [0.0233, 0.0519, 0.0464, 0.0186, 0.0108]],

        [[0.0100, 0.0242, 0.0339, 0.0536, 0.0528],
         [0.0136, 0.0331, 0.0475, 0.0614, 0.0561],
         [0.0225, 0.0278, 0.0331, 0.0550, 0.0439],
         [0.0419, 0.0456, 0.0439, 0.0469, 0.0308],
         [0.0192, 0.0325, 0.0236, 0.0458, 0.0267]]], grad_fn=<SelectBackward0>)

In [17]:
a = torch.asarray([1,2,3,4,5,6,7,8,9,0,-1,-2,-3]).reshape(1,1,1,-1)